### Prompt編集

In [ ]:
# white boy kitten character: a pure white fluffy little kitten, grey ripped t-shirt,  black smooth eyes, full body, pointing the camera with both hand, small head, crying,
# white girl kitten character: a pure white cute little kitten in a pink dress, 
# bully cat charactor: a black cat, yellow eyes, wearing a grey t-shirt and black jacket and ripped jeans, riding on single head light motorcycle
# orange kitten: The orange kitten is on the right, wearing a red outfit and holding an ice cream cone with blue and white swirls, laughing joyfully. 
# black kitten: The black kitten is on the left, wearing a blue shirt with a heart necklace. 

intro_prompt = "realistic, 4k, high resolution, real photo, "

additional_prompts = ", tall image size"

In [ ]:
def add_additional_prompts(prompt, additional_prompts):
    return f"{prompt} {additional_prompts}"

def prepend_intro_prompt(prompt, intro_prompt):
    return f"{intro_prompt} {prompt}"

final_prompt = add_additional_prompts("", additional_prompts)
final_prompt = prepend_intro_prompt(final_prompt, intro_prompt)


import ipywidgets as widgets
from IPython.display import display

# 初期のString型の変数
# initial_text = "ここに編集可能なテキストを入力してください。"
initial_text = final_prompt

# テキストエリアウィジェットを作成
text_area = widgets.Textarea(
    value=initial_text,
    placeholder='Type something',
    description='Edited Prompt:',
    disabled=False,
    layout=widgets.Layout(width='1000px', height='200px')
)

# テキストエリアを表示
display(text_area)

# テキストエリアの内容を取得する関数
def get_text_area_content():
    return text_area.value

In [ ]:
# 画像生成
from openai import OpenAI
import requests
import os
import json
import requests
import base64
import io
from PIL import Image
from IPython.display import display
from IPython.display import HTML

# テキストエリアの内容を取得して変数に格納
updated_text = get_text_area_content()

client = OpenAI()
client.api_key = os.environ['OPENAI_API_KEY']  # 環境変数から取得
def generate_image_DALLE(prompt):
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        # size="1024x1024",
        size="1024x1792", # →作れる時もあれば作れない時もある
        # size="1792x1024", # →作れる時もあれば作れない時もある
        quality="standard",
        # quality="hd",
        n=1,
    )
    return(response.data[0].url)

# Directory to save generated images
output_directory = "generated_images/"
os.makedirs(output_directory, exist_ok=True)

image_url = generate_image_DALLE(updated_text)

# Fetch the image from the URL
image_response = requests.get(image_url)
image_response.raise_for_status()  # Raise an error for bad responses

# Create an image file-like object
image_file = io.BytesIO(image_response.content)
import re

# Function to get the next available filename
def get_next_filename(base_filename, directory):
    existing_files = os.listdir(directory)
    pattern = re.compile(rf"{re.escape(base_filename)}_(\d+)\.png")
    max_index = 0
    for filename in existing_files:
        match = pattern.match(filename)
        if match:
            index = int(match.group(1))
            if index > max_index:
                max_index = index
    return f"{base_filename}_{max_index + 1}.png"

# Open and save the image
img = Image.open(image_file)
next_filename = get_next_filename("generated_image_0000", output_directory)
img.save(os.path.join(output_directory, next_filename))

# Resize and display the image in Jupyter
img = img.resize((int(1024/4), int(1792/4)))  # Resize the image to
# img = img.resize((int(1792/4), int(1024/4)))  # Resize the image to
# img = img.resize((int(1024/4), int(1024/4)))  # Resize the image to


display(img)
# display(HTML(f"<p>Saved filename: {next_filename}</p>"))

import urllib.parse

# Create a file URL for the saved image
file_url = urllib.parse.urljoin('file:', urllib.request.pathname2url(os.path.abspath(os.path.join(output_directory, next_filename))))

# Display the link with the filename
display(HTML(f'<p>Saved filename: <a href="{file_url}" target="_blank">{next_filename}</a></p>'))

promt_filename = os.path.splitext(next_filename)[0]
# Save the prompt to a file named previous_prompt.txt
with open(f"generated_images/{promt_filename}.txt", "w") as file:
    file.write(updated_text)
